# **Phân loại tin tức tiếng Việt sử dụng SVM**

In [ ]:
!pip install pyvi

## 1. Xác định bài toán

- Đầu vào bài toán: 1 đoạn tin tức tiếng Việt  
- Đầu ra: Lớp tương ứng với đoạn tin tức đó

- Dataset: VNews8td  
- Mô tả:  Được thu thập từ trang báo mạng VnExpress từ ngày 01/06/2023 - 01/06/2024. Gồm 8 lớp, chia train:val:test theo tỉ lệ 70:10:20. Mỗi văn bản sẽ gồm có tiêu đề và phần mô tả của bài báo  
- 8 lớp: 	
    * doisong (Đời sống)
	* giaoduc (Giáo dục)
	* khoahoc (Khoa học)
	* kinhte (Kinh tế)
	* suckhoe (Sức khỏe)
	* thegioi (Thế giới)
	* thethao (Thể thao)
	* thoisu (Thời sự)

### Import thư viện 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyvi import ViTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.metrics import  ConfusionMatrixDisplay

## 2. Tiền xử lý dữ liệu

In [ ]:
train_df = pd.read_csv("/kaggle/input/vnews8td/VNews8td/train.tsv", sep='\t',header=None, names=['văn bản','lớp'])
test_df = pd.read_csv("/kaggle/input/vnews8td/VNews8td/test.tsv", sep='\t',header=None, names=['văn bản','lớp'])
val_df = pd.read_csv("/kaggle/input/vnews8td/VNews8td/val.tsv", sep='\t',header=None, names=['văn bản','lớp'])

Dữ liệu bị mất cân bằng giữa các lớp --> dùng class weighted, hoặc augment data 

In [ ]:
tokens = ViTokenizer.tokenize(train_df['văn bản'][0]).split()
print(tokens)

In [ ]:
with open('/kaggle/input/vietnamese-stopwords/vietnamese_stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = list(line.strip() for line in f)

def preprocess(text):
    tokens = ViTokenizer.tokenize(text).split()
    filtered_tokens = [t for t in tokens if t not in stopwords]
    return ' '.join(filtered_tokens)

txt = preprocess(train_df['văn bản'][0])
print(txt)

Ghép các từ tiếng Việt lại và xóa stop words 

In [ ]:
train_df['processed'] = train_df['văn bản'].apply(preprocess)
val_df['processed'] = val_df['văn bản'].apply(preprocess)
test_df['processed'] = test_df['văn bản'].apply(preprocess)

Chuyển dữ liệu từ dạng văn bản về dạng số để xử lý 

In [ ]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_df['processed'])
X_val = vectorizer.transform(val_df['processed'])
X_test = vectorizer.transform(test_df['processed'])


In [ ]:
le = LabelEncoder()
y_train = le.fit_transform(train_df['lớp'])
y_val = le.transform(val_df['lớp'])
y_test = le.transform(test_df['lớp'])

In [ ]:
print("Dữ liệu training: ", X_train.shape, y_train.shape)
print("Dữ liệu validation: ", X_val.shape, y_val.shape)
print("Dữ liệu testing: ", X_test.shape, y_test.shape)

## 3. Huấn luyện mô hình & đánh giá mô hình

*Lưu ý: Đánh giá trên tập validation*

### Kernel linear với C=1

In [ ]:
import joblib
print("- Training with kernel linear ...")
print("- Train size = {}".format(X_train.shape))
model_linear = svm.SVC(kernel='linear', C=1, class_weight="balanced")
model_linear.fit(X_train, y_train)
print("- model - train complete")
output_model_filename = 'model.joblib' 
output_model_filename_vectorizer = 'vectorizer.joblib' 
output_model_filename_label_encoder = 'label_encoder.joblib' 
joblib.dump(model_linear, output_model_filename)
joblib.dump(vectorizer,output_model_filename_vectorizer)
joblib.dump(le,output_model_filename_label_encoder)
print(f"Mô hình đã được lưu thành công vào '{output_model_filename}'")


## 6. Tài liệu tham khảo

- Dataset: https://www.kaggle.com/datasets/dat111/xlnntn-th-lab08
- Vietnamese stopwords: https://github.com/stopwords/vietnamese-stopwords  
- Thư viện preprocessing tiếng Việt: pyvi - https://pypi.org/project/pyvi/